In [15]:
import json
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd


load_dotenv(override=True)


# MongoDB connection settings
mongo_uri = os.environ["DATABASE_URL"]  # Update if your MongoDB is hosted elsewhere
database_name = "cv_tech"      # Replace with your database name
collection_name = "UserActions"  # Replace with your collection name

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[database_name]
collection = db[collection_name]


## session_id

In [ ]:
pipeline_for_session_id = [
    {"$sort": {"session_id": 1, "date": 1}},
    {
        "$group": {
            "_id": "$session_id",
            "count": {"$sum": 1},
            "actions": {"$push": {"action": "$action", "date": "$date"}},
        }
    },
    {
        "$project": {
            "_id": 1,
            "actions": {"$sortArray": {"input": "$actions", "sortBy": {"date": 1}}},
        }
    },
    {"$unwind": "$actions"},
    {
        "$setWindowFields": {
            "partitionBy": "$_id",
            "sortBy": {"actions.date": 1},
            "output": {
                "previousDate": {"$shift": {"output": "$actions.date", "by": -1}}
            },
        }
    },
    {
        "$addFields": {
            "timeBetweenActions": {
                "$cond": {
                    "if": {"$eq": ["$previousDate", None]},
                    "then": None,
                    "else": {
                        "$divide": [
                            {
                                "$subtract": [
                                    {"$toDate": "$actions.date"},
                                    {"$toDate": "$previousDate"},
                                ]
                            },
                            1000,
                        ]
                    },
                }
            }
        }
    },
    {
        "$addFields": {
            "concatenatedString": {
                "$concat": [
                    {
                        "$cond": {
                            "if": {"$eq": ["$timeBetweenActions", None]},
                            "then": "_ ",
                            "else": {"$toString": "$timeBetweenActions"},
                        }
                    },
                    " ",
                    "$actions.action",
                ]
            }
        }
    },
    {
        "$group": {
            "_id": "$_id",
            "actions": {"$push": {"concatenatedString": "$concatenatedString"}},
        }
    },
    {
        "$project": {
            "_id": "$_id",
            "test": {
                "$map": {
                    "input": "$actions",
                    "as": "val",
                    "in": "$$val.concatenatedString",
                }
            },
        }
    },
]

In [ ]:
results = list(collection.aggregate(pipeline_for_session_id))

for result in results:
    print(result)

{'_id': '0d4f8akv8q18', 'test': ['_  Click on MENU_Actas del Concejo de Quito', '3.984 Click on MES_Agosto button', '0.654 Click on RESUMEN_acta_077', '9.094 Click on MENU_Actas del Concejo de Quito', '1.031 Click on MES_Enero button', '1.427 Click on RESUMEN_acta_046', '5030.329 Click on MENU_Actas del Concejo de Quito', '2.562 Click on MES_Agosto button', '78.294 Click on MES_Agosto button', '2.15 Click on MES_Enero button', '3.818 Click on RESUMEN_acta_045', '64620.725 Click on MENU_Actas del Concejo de Quito', '3.927 Click on MENU_Actas del Concejo de Quito', '9.071 Click on MES_Enero button', '11.324 Click on RESUMEN_acta_045', '2416.627 Click on MES_Enero button', '1.433 Click on RESUMEN_acta_045', '127.013 Click on MENU_Actas del Concejo de Quito', '3.598 Click on MES_Enero button', '0.886 Click on RESUMEN_acta_045', '479.616 Click on MES_Enero button', '2.614 Click on RESUMEN_acta_045', '6.332 Click on MES_Enero button', '0.863 Click on RESUMEN_acta_045', '23.762 Click on MENU_

In [17]:
user1_actions = results[0]["test"]
user4_actions = results[3]["test"]

print(user4_actions)

['_  Click on MENU_Actas del Concejo de Quito', '3.045 Click on MES_Enero button', '1.117 Click on RESUMEN_acta_045', '10.84 Click on MES_Enero button', '0.779 Click on RESUMEN_acta_045', '16.758 Click on MES_Enero button', '0.606 Click on RESUMEN_acta_045', '849.118 Click on MENU_Actas del Concejo de Quito', '44.074 Click on MENU_Actas del Concejo de Quito', '1.191 Click on MES_Enero button', '2.158 Click on RESUMEN_acta_045', '764.309 Click on MENU_Actas del Concejo de Quito', '2.987 Click on MES_Enero button', '1.487 Click on RESUMEN_acta_046', '1129.612 Click on MES_Enero button', '0.624 Click on RESUMEN_acta_046', '819.923 Click on MENU_Actas del Concejo de Quito', '3.842 Click on MES_Marzo button', '4.751 Click on MES_Marzo button', '0.975 Click on MES_Febrero button', '1.198 Click on MES_Enero button', '1.08 Click on MES_Febrero button', '0.496 Click on MES_Marzo button', '5.783 Click on RESUMEN_acta_051']


In [58]:
separated = {}

for item in user1_actions:
    string_sep = item.split(" ",1)
    key = string_sep[1]
    value = string_sep[0]
    separated[key] = value

print(separated)
    

{' Click on MENU_Actas del Concejo de Quito': '_', 'Click on MES_Agosto button': '78.294', 'Click on RESUMEN_acta_077': '0.654', 'Click on MENU_Actas del Concejo de Quito': '23.762', 'Click on MES_Enero button': '12.435', 'Click on RESUMEN_acta_046': '1.427', 'Click on RESUMEN_acta_045': '0.805'}


In [97]:
user1_df = pd.DataFrame.from_dict(separated, orient='index', columns=['Value']).reset_index()
user1_df.columns=['action', 'time']
user1_df['time'] = pd.to_numeric(user1_df['time'], errors='coerce')
user1_df

,action,time
0,Click on MENU_Actas del Concejo de Quito,NaN
1,Click on MES_Agosto button,78.294
2,Click on RESUMEN_acta_077,0.654
3,Click on MENU_Actas del Concejo de Quito,23.762
4,Click on MES_Enero button,12.435
5,Click on RESUMEN_acta_046,1.427
6,Click on RESUMEN_acta_045,0.805


In [98]:
## extra_data
user1_df.loc[7] = ['Click on PDF_acta_077', 35.68]


In [99]:
user1_df.loc[8] = ['Click on MENU_Actas del Concejo de Quito', 4857.523]
user1_df.loc[9] = ['Click on MES_Agosto button', 78.294]
user1_df.loc[10] = ['Click on PDF_acta_077', 5.0]
user1_df.loc[11] = ['Click on MES_Enero button', 765.0]
user1_df


,action,time
0,Click on MENU_Actas del Concejo de Quito,NaN
1,Click on MES_Agosto button,78.294
2,Click on RESUMEN_acta_077,0.654
3,Click on MENU_Actas del Concejo de Quito,23.762
4,Click on MES_Enero button,12.435
5,Click on RESUMEN_acta_046,1.427
6,Click on RESUMEN_acta_045,0.805
7,Click on PDF_acta_077,35.680
8,Click on MENU_Actas del Concejo de Quito,4857.523
9,Click on MES_Agosto button,78.294


In [74]:
## correct_flow
task_sucess_flow_MENU =['Click on MENU_Actas del Concejo de Quito', 'Click on MES_Agosto button', 'Click on RESUMEN_acta_077', 'Click on PDF_acta_077']
task_sucess_flow_BUTTON =['Click on LEER_ resúmenes', 'Click on MES_Agosto button', 'Click on RESUMEN_acta_077', 'Click on PDF_acta_077']


In [126]:
index_sucess = user1_df.loc[user1_df['action'] == 'Click on PDF_acta_077'].index[0]
print(index_sucess)

task_completed_df = user1_df.loc[:index_sucess]

print(task_completed_df)

7
                                      action    time
0   Click on MENU_Actas del Concejo de Quito     NaN
1                 Click on MES_Agosto button  78.294
2                  Click on RESUMEN_acta_077   0.654
3   Click on MENU_Actas del Concejo de Quito  23.762
4                  Click on MES_Enero button  12.435
5                  Click on RESUMEN_acta_046   1.427
6                  Click on RESUMEN_acta_045   0.805
7                      Click on PDF_acta_077  35.680


### Task Sucess Rate

In [127]:
action_list = task_completed_df['action'].tolist()

if set(task_sucess_flow_MENU).issubset(set(action_list )):
    print("sucess by menu")
    task_sucess_flag = True
elif set(task_sucess_flow_BUTTON).issubset(set(action_list )): 
    print("sucess by button")
    task_sucess_flag = True
else:
    print("not-sucess")
    task_sucess_flag = False

print(task_sucess_flag)

sucess by menu
True


##### extract sucess only list

In [116]:
extract_until_success = []
for item in action_list:
    extract_until_success.append(item)
    if item == 'Click on PDF_acta_077':
        break
    
print(extract_until_success)



[' Click on MENU_Actas del Concejo de Quito', 'Click on MES_Agosto button', 'Click on RESUMEN_acta_077', 'Click on MENU_Actas del Concejo de Quito', 'Click on MES_Enero button', 'Click on RESUMEN_acta_046', 'Click on RESUMEN_acta_045', 'Click on PDF_acta_077']


### User Error Rate

In [129]:
if (action_list == task_sucess_flow_MENU) or (action_list == task_sucess_flow_BUTTON):
    print("User correctly completed the task")
    user_error_flag = False
else:
    print("User ERROR")
    user_error_flag = True

print(user_error_flag)

User ERROR
True


### Time on Task

In [128]:
task_completed_df['time'].sum()

153.057

## pipeline for user_id

In [12]:
pipeline_for_user_id = [
    {"$sort": {"user_id": 1, "date": 1}},
    {
        "$group": {
            "_id": "$user_id",
            "count": {"$sum": 1},
            "actions": {"$push": {"action": "$action", "date": "$date"}},
        }
    },
    {
        "$project": {
            "_id": 1,
            "actions": {"$sortArray": {"input": "$actions", "sortBy": {"date": 1}}},
        }
    },
    {"$unwind": "$actions"},
    {
        "$setWindowFields": {
            "partitionBy": "$_id",
            "sortBy": {"actions.date": 1},
            "output": {
                "previousDate": {"$shift": {"output": "$actions.date", "by": -1}}
            },
        }
    },
    {
        "$addFields": {
            "timeBetweenActions": {
                "$cond": {
                    "if": {"$eq": ["$previousDate", None]},
                    "then": None,
                    "else": {
                        "$divide": [
                            {
                                "$subtract": [
                                    {"$toDate": "$actions.date"},
                                    {"$toDate": "$previousDate"},
                                ]
                            },
                            1000,
                        ]
                    },
                }
            }
        }
    },
    {
        "$addFields": {
            "concatenatedString": {
                "$concat": [
                    {
                        "$cond": {
                            "if": {"$eq": ["$timeBetweenActions", None]},
                            "then": "_ ",
                            "else": {"$toString": "$timeBetweenActions"},
                        }
                    },
                    " ",
                    "$actions.action",
                ]
            }
        }
    },
    {
        "$group": {
            "_id": "$_id",
            "actions": {"$push": {"concatenatedString": "$concatenatedString"}},
        }
    },
    {
        "$project": {
            "_id": "$_id",
            "test": {
                "$map": {
                    "input": "$actions",
                    "as": "val",
                    "in": "$$val.concatenatedString",
                }
            },
        }
    },
]

In [13]:
results_userid= list(collection.aggregate(pipeline_for_user_id))

for result in results_userid:
    print(result)

{'_id': None, 'test': ['_  Click on MENU_Actas del Concejo de Quito', '3.984 Click on MES_Agosto button', '0.654 Click on RESUMEN_acta_077', '9.094 Click on MENU_Actas del Concejo de Quito', '1.031 Click on MES_Enero button', '1.427 Click on RESUMEN_acta_046', '172.806 Click on MENU_Actas del Concejo de Quito', '4857.523 Click on MENU_Actas del Concejo de Quito', '2.562 Click on MES_Agosto button', '78.294 Click on MES_Agosto button', '2.15 Click on MES_Enero button', '3.818 Click on RESUMEN_acta_045', '64620.725 Click on MENU_Actas del Concejo de Quito', '3.927 Click on MENU_Actas del Concejo de Quito', '9.071 Click on MES_Enero button', '11.324 Click on RESUMEN_acta_045', '2416.627 Click on MES_Enero button', '1.433 Click on RESUMEN_acta_045', '127.013 Click on MENU_Actas del Concejo de Quito', '3.598 Click on MES_Enero button', '0.886 Click on RESUMEN_acta_045', '479.616 Click on MES_Enero button', '2.614 Click on RESUMEN_acta_045', '6.332 Click on MES_Enero button', '0.863 Click on